### Day 17, Part 1: Test Cases


#### Details:

- The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified active (#) or inactive (.) state

- At every integer 3-dimensional coordinate (x,y,z), there exists a single cube which is either active or inactive.

- Each cube only ever considers its neighbors: any of the 26 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3, its neighbors include the cube at x=2,y=2,z=2, the cube at x=0,y=2,z=3, and so on.

- During a cycle, all cubes simultaneously change their state according to the following rules:

```If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.```

In [3]:
filepath = "day17_test_data.txt"
with open(filepath) as fh:
    lines = [line.strip() for line in fh.readlines()]

In [141]:
# get it into a matrix 
import numpy as np

matrix_in = [list(code) for code in lines]

# matrix form of z = 0 -> we assume the surroundings are all inactive at the start
matrix = np.array([np.array(row) for row in matrix_in])

# going to switch to 1s and 0s
# 1 == Active, 0 == Inactive -> will let me sum
matrix[matrix == '#'] = 1
matrix[matrix == '.'] = 0
z0 = matrix.astype(int)
print(z0)

[[0 1 0]
 [0 0 1]
 [1 1 1]]


In [142]:
# we now need a way to expand these outward in all directions.
# initially we have all 0s for z = -1 & z = 1 
# if something does not exist, we can just assume it is 0
z1 = np.full(z0.shape, 0)
zn1 = np.full(z0.shape, 0)

In [143]:
array3d = np.stack((z1, z0, zn1))
print(array3d)

[[[0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 1 0]
  [0 0 1]
  [1 1 1]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]]]


### Need to Expand Grid Out:

- Just going to expand early....should really be a 5 x 5....or more...


In [150]:
# can we pad a 3D object? Oh wow....we can! Let's pad this up!
final_matrix = np.pad(array3d, pad_width=10, mode='constant', constant_values=0)
print(f"New shape is: {final_matrix.shape}")

New shape is: (23, 23, 23)


### Building Functions for iterating over each Val: 

- Assumption is that non-existent coordinate actually should exist, but would start as a 0.
- However, it will need to be constantly expanded (no idea how to do this)


#### What should be surrounding (1,1,1) ?

- x=1,y=2,z=3:
- Inspiration: https://stackoverflow.com/questions/2373306/pythonic-and-efficient-way-of-finding-adjacent-cells-in-grid


In [151]:
# testing 
x_in, y_in, z_in = 1,1,1
test_run = []
for x,y,z in [(x_in + i, y_in + j, z_in + k) for i in (-1,0,1) for j in (-1,0,1) for k in (-1,0,1)]:
    if x == x_in and y == y_in and z == z_in:
        continue
    test_run.append((x,y,z))

print(len(set(test_run)))

26


In [152]:
# single func
def countNeighbors(coord, matrix):
    """Review neighbor states"""
    depth, row, col = coord
    count = 0
    for d,r,c in [(depth + i, row + j, col + k) for i in (-1,0,1) for j in (-1,0,1) for k in (-1,0,1)]:
        if d == depth and r == row and c == col:
            continue
        try:
            count += matrix[(d,r,c)]
        except:
            pass

    return count
    

In [153]:
# array copy, but all 0s:
my_copy = np.empty_like(final_matrix)

# iterate over each value
for depth in range(final_matrix.shape[0]):
    for row in range(final_matrix.shape[1]):
        for col in range(final_matrix.shape[2]):
                
                # find state & count neighbor states
                coord = (depth, row, col)
                state = final_matrix[coord]
                active_count = countNeighbors(coord, final_matrix)
                
                if state:
                    # keep active
                    if 2 <= active_count <= 3:
                        my_copy[coord] = 1
                    else:
                        my_copy[coord] = 0
                else:
                    if active_count == 3:
                        my_copy[coord] = 1
                    else:
                        my_copy[coord] = 0

# update
final_matrix = np.array(my_copy, copy=True)


# Compare to Test Case
print("After 1 Cycle")

# check z = -1
print("Check z = -1")
print(final_matrix[11])

# print z = 0 (this is the middle value...)
# wow, this is correct...but it is going to get big 
print("Check z = 0")
print(final_matrix[12])

print("Check z = 1")
print(final_matrix[13])

After 1 Cycle
Check z = -1
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

### Iterate 6 times & sum all 1s

In [157]:
# Just going to pad in each loop
final_matrix = np.pad(array3d, pad_width=1, mode='constant', constant_values=0)
print(f"New shape is: {final_matrix.shape}")

for _ in range(6):

    # array copy, but all 0s:
    my_copy = np.empty_like(final_matrix)

    # iterate over each value
    for depth in range(final_matrix.shape[0]):
        for row in range(final_matrix.shape[1]):
            for col in range(final_matrix.shape[2]):

                    # find state & count neighbor states
                    coord = (depth, row, col)
                    state = final_matrix[coord]
                    active_count = countNeighbors(coord, final_matrix)

                    if state:
                        # keep active
                        if 2 <= active_count <= 3:
                            my_copy[coord] = 1
                        else:
                            my_copy[coord] = 0
                    else:
                        if active_count == 3:
                            my_copy[coord] = 1
                        else:
                            my_copy[coord] = 0

    # update
    final_matrix = np.array(my_copy, copy=True)
    
    # Add padding 
    final_matrix = np.pad(final_matrix, pad_width=1, mode='constant', constant_values=0)
    print(f"New shape is: {final_matrix.shape}")
    
# print active:
print(f"Total active: {np.sum(final_matrix)}")

New shape is: (5, 5, 5)
New shape is: (7, 7, 7)
New shape is: (9, 9, 9)
New shape is: (11, 11, 11)
New shape is: (13, 13, 13)
New shape is: (15, 15, 15)
New shape is: (17, 17, 17)
Total active: 112
